In [1]:
from dotenv import load_dotenv

import os

GEMINI_API_KEY = os.environ.get("GEMINI_KEY_API")

UPSTASH_VECTOR_REST_URL = os.environ.get("UPSTASH_VECTOR_REST_URL")
                                         
UPSTASH_VECTOR_REST_TOKEN = os.environ.get("UPSTASH_VECTOR_REST_TOKEN")

load_dotenv()

True

In [32]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY



embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [33]:
from langchain_community.vectorstores import UpstashVectorStore

store = UpstashVectorStore(
    embedding=embeddings,
    index_url = UPSTASH_VECTOR_REST_URL,
    index_token = UPSTASH_VECTOR_REST_TOKEN 
)

In [34]:
retriever = store.as_retriever(
    search_type = "similarity",
    search_kwargs = {'k':2}
)

In [35]:
retriever.invoke("what is the city name after trees")

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Boise,_Idaho', 'title': 'Boise, Idaho'}, page_content=' landmark, an oasis dominated by cottonwood trees. This led the French trappers to call the area "la rivière boisée" ("the wooded river").\n\n\n==== Pronunciation ====\nMost local and longtime residents use the pronunciation /ˈbɔɪsiː/ (BOY-see), as given on the city\'s website. The pronunciation is sometimes used as a shibboleth, as outsiders (and newcomers) tend to mispronounce the city\'s name as /ˈbɔ'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Boise,_Idaho', 'title': 'Boise, Idaho'}, page_content='-speaking guide, overwhelmed by the sight of the verdant river, yelled "Les bois! Les bois!" ("The woods! The woods!")—and the name stuck.\nThe name may also derive from earlier mountain men who named the river that flows through the city. In the 1820s, French Canadian fur trappers associated with the British-owned Hudson\'s Bay Company set trap lines in t

In [36]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

In [37]:
LLM_CONFIG = {
    "model": "gemini-1.5-pro",
    "google_api_key": GEMINI_API_KEY

}

llm = GoogleGenerativeAI(**LLM_CONFIG)

In [51]:
from langchain_core.prompts import ChatPromptTemplate

message = '''
Answer this question using the provided context only.

{question}

Context:
{context}
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "if the user is greeting then respond to the greeting"),
        ("human", message)
    ]
)

In [52]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    modified = lambda x: x["num"] + 1
)

parser = StrOutputParser()

In [53]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | parser

In [67]:
chain.invoke("Hope you're fine?")

'I am doing well, thank you for asking.\n'